In [0]:
import pandas as pd
import pymc as pm
import arviz as az

In [20]:
### with the new combination of theano and PyMC3, amongst others, DelayedSaturedMMM is not working
from pymc_marketing.mmm import DelayedSaturatedMMM

ImportError: cannot import name 'Image' from 'PIL' (unknown location)

In [18]:
data = pd.read_csv(
    'https://raw.githubusercontent.com/Garve/datasets/4576d323bf2b66c906d5130d686245ad205505cf/mmm.csv',
    parse_dates=['Date']
)

In [19]:
mmm = DelayedSaturatedMMM(
    data=data,
    target_column="Sales",
    date_column="Date",
    channel_columns=["TV", "Radio", "Banners"],
)

NameError: name 'DelayedSaturatedMMM' is not defined

In [ ]:
pm.model_to_graphviz(model=mmm.model)

In [ ]:
mmm.fit()

In [ ]:
az.summary(
    data=mmm.fit_result,
    var_names=["intercept", "beta_channel", "alpha",  "lam", "sigma"]
)

In [ ]:
mmm.plot_channel_parameter(param_name="alpha", figsize=(9, 5))

In [ ]:
mmm.plot_channel_contribution_share_hdi()

In [ ]:
mmm.plot_posterior_predictive(original_scale=True)

In [ ]:
mmm.plot_components_contributions()